In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

from APP_model import *

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ESSAYS_PATH = 'datasets/essays.csv'
MBTI_PATH = 'datasets/mbti_1.csv'

In [3]:
def load_data(essays_path, mbti_path):
    essays_df = pd.read_csv(ESSAYS_PATH, encoding='latin1')
    cat_columns = ['cEXT', 'cNEU', 'cAGR', 'cCON', 'cOPN']
    essays_df[cat_columns] = essays_df[cat_columns].replace({'y': 1, 'n': 0})

    mbti_df = pd.read_csv(MBTI_PATH)
    types = mbti_df['type'].unique()
    types = dict(zip(types, range(len(types))))
    mbti_df['type'] = mbti_df['type'].replace(types)

    return essays_df, mbti_df

data = load_data(ESSAYS_PATH, MBTI_PATH)

In [4]:
import json
config = json.load(open('config.json'))

In [5]:
model = APP(config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [6]:
cat_columns = ['cEXT', 'cNEU', 'cAGR', 'cCON', 'cOPN']

running_loss_essay = 0
running_loss_mbti = 0

In [9]:
essays_df, mbti_df = data

In [11]:
tokeniser = BertTokenizer.from_pretrained('bert-base-uncased')

In [14]:
for index, row in essays_df.iterrows():
    text = row['TEXT']
    labels = row[cat_columns].to_dict()
    
    # optimizer.zero_grad()

    preds = model(tokeniser(text, padding=True, truncation=True, return_tensors='pt'))
    loss = model.get_ocean_loss(preds, labels)
    print(loss)
    # # loss.backward()
    # # optimizer.step()

    # running_loss_essay += loss.item()

    if index % 1 == 0:    # print every 2000 mini-batches
        print(f'essay index: {index + 1} loss: {running_loss_essay / 100:.3f}')
        running_loss_essay = 0.0

tensor(3.3418, grad_fn=<AddBackward0>)
essay index: 1 loss: 0.000
tensor(3.5937, grad_fn=<AddBackward0>)
essay index: 2 loss: 0.000
tensor(3.3384, grad_fn=<AddBackward0>)
essay index: 3 loss: 0.000


KeyboardInterrupt: 